# Ejercicios

## Librerías

Importaremos todas las librerías que se usaran durante toda la tarea.

In [246]:
import sys
import numpy as np
import pandas as pd
from pprint import pprint

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from sklearn.decomposition import PCA
from sklearn import manifold
from sklearn_som.som import SOM
from sklearn.manifold import TSNE

## T-SNE

### Problema

Usamos para T-SNE la divergencia de Kullback-Leibler. Para distribuciones discretas su definición es: $d\left(P^1,P^2\right)=\sum_i P^1_i\log{\frac{P^1_i}{P^2_i}}$.

Calcula si $d\left(P^1,P^2\right)$ si $P^1\sim Bern\left(\theta_1\right)$ y $P^2\sim Bern\left(\theta_2\right)$. Para $\theta_1$ fija, grafica $d\left(P^1,P^2\right)$ como función de $\theta_2$, verifica que efectivamente mide de alguna manera la disimilitud entre $P^1$ y $P^2$.

### Solución

Recordemos que si $X\sim Bern\left(\theta\right)$ entonces $X\in\{0,1\}$ y con función de masa de probabilidad

$$f\left(x;\theta\right)=\theta^x\left(1-\theta\right)^{1-x}.$$

Por lo que $d\left(P^1,P^2\right)$ si $P^1\sim Bern\left(\theta_1\right)$ y $P^2\sim Bern\left(\theta_2\right)$ es

$$d\left(P^1,P^2\right)=\sum_i P^1_i\log{\frac{P^1_i}{P^2_i}}=\theta_1^0\left(1-\theta_1\right)^1\log\left(\frac{\theta_1^0\left(1-\theta_1\right)^1}{\theta_2^0\left(1-\theta_2\right)^1}\right)+\theta_1^1\left(1-\theta_1\right)^0\log\left(\frac{\theta_1^1\left(1-\theta_1\right)^0}{\theta_2^1\left(1-\theta_2\right)^0}\right)=\left(1-\theta_1\right)\log\left(\frac{\left(1-\theta_1\right)}{\left(1-\theta_2\right)}\right)+\theta_1\log\left(\frac{\theta_1}{\theta_2}\right).$$

Ahoea veamos a $d\left(P^1,P^2\right)$ como una función de $\theta_2$ para un $\theta_1$ fijo:

In [247]:
d = 2
t = 10**2
x = np.linspace(1, 0, t, endpoint=False)[::-1]
z = np.linspace(1, 0, 50, endpoint=False)[::-1]
x=x[:-1]
z=z[:-1]
x,z = np.meshgrid(x,z)
y = (1-z)*np.log((1-z)/(1-x))+z*np.log(z/x)
x=x.flatten()
y=y.flatten()
z=z.flatten()
x=x.round(d)
z=z.round(d)
df = pd.DataFrame({'x':x, 'y':y, 'z':z})
fig1 = px.line(df, x='x', y='y', animation_frame='z')

x = np.linspace(1, 0, 50, endpoint=False)[::-1]
z = np.linspace(1, 0, 50, endpoint=False)[::-1]
x=x.flatten()
z=z.flatten()
y=np.zeros(x.size)
df = pd.DataFrame({'x':x, 'y':y, 'z':z})
fig2 = px.scatter(df, x='x', y='y', animation_frame='z', color_discrete_sequence=['orange'])
fig2.update_traces(marker_size=10)

fig = go.Figure(
    data=fig1.data + fig2.data,
    frames=[
        go.Frame(data=fr1.data + fr2.data, name=fr1.name)
        for fr1, fr2 in zip(fig1.frames, fig2.frames)
    ],
    layout=fig1.layout,
)


fig.update_layout(
        template="simple_white",
        title='Divergencia de Kullback-Leible para Bernoullis',
        width=1000,
        height=1000,
        sliders=[{"transition":dict(duration=0.01),'currentvalue':{"prefix": r"\theta_2 = "},}]
    )
fig.update_xaxes(title=r'\theta_1', showgrid=True, showline=False, zeroline=True)
fig.update_yaxes(title=r'd', showgrid=True, showline=True, zeroline=True)

fig.show()


Podemos ver que la gráfica alcanza un mínimo es alcanzado cuando $\theta_1=\theta_2$ así como mientras más estos se alejan de este punto la $d$ incrementa, es decir mientras más distintos sean $\theta_1$ y $\theta_2$ mayor aumenta la distancia de las variables aleatorias. Por lo tanto concluimos que la divergencia de Kullback-Leible mide la similitudes que hay entre dos variable aleatorias con distirbución Bernoulli.

## Algoritmos de Reducción de dimensionalidad

### Problema

En este ejercicio hacemos una exploración de indicadores de pobreza a nivel municipal usando datos del INEGI del 2020.

La idea es aprovechar los métodos vistos hasta ahora (PCA, ISOMAP, SOM, T-SNE, ...) para poder entender mejor diferencias en pobreza entre municipios. Se puede limitar el análisis a un(os) estado(s).

El resultado debe ser un reporte con gráficas y resúmenes informativos, integrados en un texto. Incorpora una compración entre la fortalezas y debilidades de los diferentes métodos usados.

### Solución

#### Lectura de Datos

Se leen los datos a partir del excel "Indice de marginación por municipio 2020" proporcionado por la CONAPO. A su vez se filtran estos para el analisis de los estaods Monterrey, Guanajuato y Chiapas; esto para tener un representante en las regiones norte, centro y sur del país respectivamente. Por último usamos como índices a las claves de los estados y municipios proporcionados en el mismo excel como índices.

In [248]:
source_data = pd.read_excel('IMM_2020.xls', sheet_name="IMM_2020")
ID_STATE = [19, 11, 7]
source_data = source_data[ source_data.CVE_ENT.isin(ID_STATE) ]
source_data.set_index(["CVE_ENT", "CVE_MUN"], inplace = True)           
source_data

NOM_ENT         NOM_MUN  POB_TOT      ANALF      SBASC  \
CVE_ENT CVE_MUN                                                              
7       7001        Chiapas      Acacoyagua    17994  10.762105  51.759328   
        7002        Chiapas           Acala    21187  15.883866  50.666667   
        7003        Chiapas      Acapetahua    26899  13.733270  50.582072   
        7004        Chiapas      Altamirano    36160  22.993684  62.952005   
        7005        Chiapas          Amatán    24512  17.084370  52.222718   
...                     ...             ...      ...        ...        ...   
19      19047    Nuevo León         Hidalgo    16086   2.878951  24.660848   
        19048    Nuevo León  Santa Catarina   306322   1.548953  18.989656   
        19049    Nuevo León        Santiago    46784   1.980500  27.059032   
        19050    Nuevo León      Vallecillo     1552   4.668305  52.252252   
        19051    Nuevo León      Villaldama     3573   3.012048  32.461902   

                    OVSDE     OVSEE      OVSAE       OVPT       VHAC  \
CVE_ENT CVE_MUN                                                        
7       7001     3.281789  1.707643   6.608077   4.071643  37.967793   
        7002     2.065274  1.056321   3.415281   6.318981  45.663029   
        7003     2.881489  0.811003  16.664798   8.162350  29.858820   
        7004     1.935814  3.607266  16.001023   7.584070  34.881321   
        7005     1.314496  4.594595  11.113841  25.741196  45.827438   
...                   ...       ...        ...        ...        ...   
19      19047    0.292416  0.267529   0.983015   0.292434  17.158470   
        19048    0.027691  0.065599   0.255152   0.854292  12.172526   
        19049    0.331277  0.265836   1.721245   1.523208  15.437043   
        19050    1.806452  0.451613   2.322581   0.193548  12.811388   
        19051    0.756514  0.308210   1.709162   0.700476  17.549408   

                    PL.5000      PO2SM    IM_2020   GM_2020  IMN_2020  
CVE_ENT CVE_MUN                                                        
7       7001      55.029454  89.111184  53.290790     Medio  0.834460  
        7002      29.197149  85.024014  53.300950     Medio  0.834619  
        7003      76.776088  88.608530  52.149949      Alto  0.816596  
        7004      70.636062  92.028986  49.784066      Alto  0.779549  
        7005     100.000000  96.062176  48.759325      Alto  0.763503  
...                     ...        ...        ...       ...       ...  
19      19047      0.180281  52.933289  59.783205  Muy bajo  0.936122  
        19048      0.741050  45.969352  60.716763  Muy bajo  0.950740  
        19049      8.047623  42.458426  60.034629  Muy bajo  0.940059  
        19050    100.000000  76.205288  56.166783      Bajo  0.879494  
        19051    100.000000  77.128954  56.859541  Muy bajo  0.890341  

[221 rows x 15 columns]

#### Filtración de Datos

Puesto que tanto el numbre del estado y nombre del municipio no proporcionan más que un identificador para el analisis de los datos, entonces las ignoramos. Puesto que todos nuestros datos ya están en proporción al total de la población, entonces podemos precindir de el. Para evitar el cesgo de nuestro analisis con los indices empíricos de marginación también no haremos uso de ellos, además de que está información ya está considerada en las demás características. Por último normalizamos nuestros datos para la mejor manipulación de ellos

In [249]:
data = source_data.drop(['NOM_ENT','NOM_MUN','POB_TOT','IM_2020','IMN_2020','GM_2020'], axis=1)
data_norm = (data - data.mean())/data.std()
data_norm

ANALF     SBASC     OVSDE     OVSEE     OVSAE      OVPT  \
CVE_ENT CVE_MUN                                                               
7       7001    -0.104157  0.371383  0.201831  0.298969 -0.079842 -0.534259   
        7002     0.546369  0.294003 -0.113280 -0.143299 -0.420239 -0.267299   
        7003     0.273217  0.288013  0.098142 -0.309879  0.992346 -0.048326   
        7004     1.449402  1.164020 -0.146813  1.588876  0.921578 -0.117019   
        7005     0.698847  0.404199 -0.307751  2.259305  0.400536  2.039857   
...                   ...       ...       ...       ...       ...       ...   
19      19047   -1.105414 -1.547664 -0.572498 -0.678915 -0.679552 -0.983189   
        19048   -1.274340 -1.949284 -0.641069 -0.816032 -0.757153 -0.916446   
        19049   -1.219529 -1.377831 -0.562432 -0.680065 -0.600847 -0.836986   
        19050   -0.878145  0.406291 -0.180322 -0.553916 -0.536736 -0.994936   
        19051   -1.088509 -0.995213 -0.452284 -0.651292 -0.602135 -0.934718   

                     VHAC   PL.5000     PO2SM  
CVE_ENT CVE_MUN                                
7       7001     0.501790 -0.320465  0.519453  
        7002     1.059448 -1.097715  0.245173  
        7003    -0.085850  0.333853  0.485721  
        7004     0.278120  0.149110  0.715259  
        7005     1.071362  1.032620  0.985916  
...                   ...       ...       ...  
19      19047   -1.006218 -1.970782 -1.908353  
        19048   -1.367539 -1.953909 -2.375685  
        19049   -1.130966 -1.734067 -2.611294  
        19050   -1.321242  1.032620 -0.346629  
        19051   -0.977888  1.032620 -0.284644  

[221 rows x 9 columns]

#### Gráfica de proyecciones y componentes con datos agrupados por Estado

Recordemos que PCA, ISOMAP y T-SNE harán proyecciones de nuestros datos en menor dimensión. Luego intentaremos ver que relación hay entre los estados seleccionados y las carácteristicas respectivas de los algoritmos, para ello haremos una función para graficar sus proyecciones y componentes.

In [250]:
def porj_comp_plot_by_state(df, proj,coeff=None,labels=None,title="Grafica"):
    
    x = proj[:,0]
    y = proj[:,1]
    
    x /= (x.max() - x.min())
    y /= (y.max() - y.min())

    if labels is None and coeff is not None:
        labels = ["Var"+str(i+1) for i in range(coeff.shape[0])]
    state_ids = np.array(df.index.values.tolist())[:, 0]

    fig = go.Figure()
    for state_id in ID_STATE:
        fig.add_scatter(
            x = x[state_ids == state_id],
            y = y[state_ids == state_id],
            name = df.loc[state_id].NOM_ENT.iloc[0],
            mode="markers"
        )

    if coeff is not None:
        for direc,label in zip(coeff,labels):
            fig.add_trace(
                go.Scatter(
                    x = [0,direc[0]],
                    y = [0,direc[1]],
                    name = label
                )
            )
    
    fig.update_layout(
        template="simple_white",
        title=title,
        width=1000,
        height=1000
    )
    fig.update_xaxes(title='PC1', range=[-1,1], showgrid=True, showline=True, zeroline=False)
    fig.update_yaxes(title='PC2', range=[-1,1], showgrid=True, showline=True, zeroline=False)

    fig.show()

#### Gráfica de proyecciones y componentes con datos agrupados por índice de marginación

Recordemos que PCA, ISOMAP y T-SNE harán proyecciones de nuestros datos en menor dimensión. Luego intentaremos ver que relación hay entre el índice de marginación y las carácteristicas respectivas de los algoritmos, para ello haremos una función para graficar sus proyecciones y componentes.

In [251]:
def porj_comp_plot_by_imn(df_source, df, proj,coeff=None,labels=None,title="Grafica"):
    
    x = proj[:,0]
    y = proj[:,1]
    
    x /= (x.max() - x.min())
    y /= (y.max() - y.min())

    fig = make_subplots(rows=3, cols=3, subplot_titles=data_norm.columns)
    
    for i,carac in enumerate(data_norm.columns):
        color = np.array(df[carac].values.tolist())
        color -= color.min()
        color /= (color.max() - color.min())
        fig.add_scatter(
            row=int(i/3)+1,
            col=int(i%3)+1,
            x = x,
            y = y,
            marker=dict(
                color=color,
                coloraxis="coloraxis"
            ),
            showlegend=False,
            mode="markers"
        )
        if coeff is not None:
            fig.add_trace(
                    go.Scatter(
                        x = [0,coeff[i,0]],
                        y = [0,coeff[i,1]],
                        name = labels[i]
                    ),
                    row=int(i/3)+1,
                    col=int(i%3)+1
                )
    fig.update_layout(
        template="simple_white",
        coloraxis=dict(colorscale='Viridis'),
        title=title,
        width=1000,
        height=1000
    )
    fig.update_xaxes(title='PC1', range=[-1,1], showgrid=True, showline=True, zeroline=False)
    fig.update_yaxes(title='PC2', range=[-1,1], showgrid=True, showline=True, zeroline=False)
    fig.show()

    if labels is None and coeff is not None:
        labels = ["Var"+str(i+1) for i in range(coeff.shape[0])]
    imn = np.array(df_source.IMN_2020.values.tolist())

    fig = go.Figure()
    fig.add_scatter(
        x = x,
        y = y,
        marker=dict(
            color=imn,
            colorbar=dict(
                title="IMN",
                len=0.6,
                yanchor="bottom",
                y = 0.1
            ),
            colorscale="Viridis"
        ),
        showlegend=False,
        mode="markers"
    )

    if coeff is not None:
        for direc,label in zip(coeff,labels):
            fig.add_trace(
                go.Scatter(
                    x = [0,direc[0]],
                    y = [0,direc[1]],
                    name = label,
                )
            )
    
    fig.update_layout(
        template="simple_white",
        title=title,
        width=1000,
        height=1000
    )
    fig.update_xaxes(title='PC1', range=[-1,1], showgrid=True, showline=True, zeroline=False)
    fig.update_yaxes(title='PC2', range=[-1,1], showgrid=True, showline=True, zeroline=False)
    
    fig.show()

### PCA

Primero encontraremos las proyecciones de nuestro datos y direcciones componentes del PCA para los datos a analizar.

In [252]:
pca = PCA(2)
proj = pca.fit_transform(data_norm)
np.set_printoptions(threshold=sys.maxsize)
pca.components_

array([[ 0.39367314,  0.37551806,  0.21966112,  0.30654071,  0.27592212,
         0.37374742,  0.37671555,  0.2565919 ,  0.37177313],
       [-0.11542743, -0.26067563,  0.53107387,  0.45280442,  0.45477745,
         0.05697476, -0.17415363, -0.31732768, -0.30092926]])

Recordemos que los significados de las nomenclaturas para el analisis de nuestros datos:

- CVE_MUN  :  Clave del municipio
- NOM_MUN  :  Nombre del municipio
- POB_TOT  :  Población total
- ANALF    :  % Población de 15 años o más analfabeta 
- SBASC	   :  % Población  de 15 años o más sin educación básica
- OVSDE	   :  % Ocupantes en viviendas particulares sin drenaje ni excusado
- OVSEE	   :  % Ocupantes en viviendas particulares sin energía eléctrica
- OVSAE	   :  % Ocupantes en viviendas particulares sin agua entubada
- OVPT	   :  % Ocupantes en viviendas particulares con piso de tierra
- VHAC     :  % Viviendas particulares con hacinamiento
- PL.5000  :  % Población en localidades con menos de 5 000 habitantes
- PO2SM	   :  % Población ocupada con ingresos menores a 2 salarios mínimos

##### Analisis por estados

Graficaremos las proyecciones de los datos agrupados por estados, así como las componentes respectivas a las caracteristicas de estos.

In [253]:
porj_comp_plot_by_state(source_data, proj[:,0:2], np.transpose(pca.components_[0:2,:]), data_norm.columns[:], title="PCA por estados")

Empezaremos por analizar las direcciones de nuestras componentes:

Primero notemos como OVSDE,OVSEE y OVSAE apuntan a la misma dirección, por lo que nos indican que estás características están relacionadas, lo cuál tiene sentido puesto que estas representan la carencia o no de servicios públicos, los cuales son una parte importante en el día a día; puesto que la hoy en día estos son muy necesarios en nuestro día a día no hay claro una jerarquía de ellas y por lo tanto esto influyen de una manera similar a nuestros datos. También si un dato está muy al Norte este estará fuertemente influenciado por estas tres características, es decir si un municipio presenta un alto % Ocupantes en viviendas particulares sin drenaje ni excusado, un alto % Ocupantes en viviendas particulares sin energía eléctrica o un alto % Ocupantes en viviendas particulares sin agua entubada; entonces este se ubcaría en dirección al Norte (un poco al Este). 

Ahora veamos que ANALF y SBASC apuntan ambos al Sur-Este, y parecen estar bastante juntos. Lo cuál es coherente con las definiciones de estas, ya que una persona sin educación básica es díficil que esta haya aprendido a leer y a escribir, así como una persona analfabeta es díficil que no haya ido a la escuela. También VHAC, PL.5000, PO2SM apuntas los tres hacía el Sur-Este, con PO2SM un punto intermedia de estos, y justo refleja lo que nuestras características nos dicen ya que una persona que gana poco no podrá tener una gran extensión de territorio y por lo cuál tendra que vivir en una lacación con hacinamiento, así como una persona que viva en una comunidad de pocas personas tendrá menos oportunidad de recibir un salario alto puesto que el flujo de dinero no es mucho; notese que las conversiones de estas no son verdaderas por lo que podemos el que esten algo separadas unas de otras es bueno. Un dato está muy al Sur-Este estará fuertemente influenciado por estas cinco características, es decir si un municipio presenta un alto % Población de 15 años o más analfabeta, un alto % Población  de 15 años o más sin educación básica, un alto % Viviendas particulares con hacinamiento, un alto % Población en localidades con menos de 5 000 habitantes o un alto % Población ocupada con ingresos menores a 2 salarios mínimos; entonces este se ubcaría en dirección al Sur Este (un poco al Este).

Por último tenemos a la característica OVPT, que parece este algo relacionada con ANALF, SBASC, VHAC y PO2SM puesto todos apuntan hacia el Este, ya que todos están ligeramente relacionadas por un tema económico (salarial).

Ahora analizando por estados vemos que:

Guanajuato se encuentra muy al centro hacia noroeste por lo que no presenta un alto % Población de 15 años o más analfabeta, un alto % Población  de 15 años o más sin educación básica, un alto % Viviendas particulares con hacinamiento, un alto % Población en localidades con menos de 5 000 habitantes o un alto % Población ocupada con ingresos menores a 2 salarios mínimos; pero si podria mostrar un significativo % Ocupantes en viviendas particulares sin drenaje ni excusado, un significativo % Ocupantes en viviendas particulares sin energía eléctrica o un significativo % Ocupantes en viviendas particulares sin agua entubada ya que estan apuntando hacia el norte estos municipios. Esto es medida controlada puesto que parecen ser estar muy condensados en el origen. Lo que habla de un Estado muy neutral

Nuevo León parece estar distribuido muy parecido a Guanajuato, con excepción de que estos no están tan centrados si no más dispersos hacia el noroeste por lo que son más fuertes las acerveraciones que hicimos en Guanajuato (sobre las características). Lo que habla más de un estado con menos deficiencias.

Caso contrario a Chiapas donde podemos ver que es fuertemente influenciado por un alto % Población de 15 años o más analfabeta, un alto % Población  de 15 años o más sin educación básica, un alto % Viviendas particulares con hacinamiento, un alto % Población en localidades con menos de 5 000 habitantes o un alto % Población ocupada con ingresos menores a 2 salarios mínimos, y no tanto por un alto % Ocupantes en viviendas particulares sin drenaje ni excusado, un alto % Ocupantes en viviendas particulares sin energía eléctrica o un alto % Ocupantes en viviendas particulares sin agua entubada. Lo que habla un pais con muchos problemas económicos y educativos.

##### Analisis por índice de marginación

Graficaremos las proyecciones de los datos agrupados por índice de marginación, así como las componentes respectivas a las caracteristicas de estos.

In [254]:
porj_comp_plot_by_imn(source_data, data_norm, proj[:,0:2], np.transpose(pca.components_[0:2,:]), data_norm.columns[:], title="PCA por índice de marginación")

Ahora analizando por estados vemos que:

En cada una de las gráficas por colores (donde el color es la dimensión de su respectiva característica), que los colores sigue la direción de nuestras componentes.

Podemos observar que los índices de marginación menores están más al norte lo que nos habla que son municipios influenciados por un alto % Ocupantes en viviendas particulares sin drenaje ni excusado, un alto % Ocupantes en viviendas particulares sin energía eléctrica o un alto % Ocupantes en viviendas particulares sin agua entubada. Así como municipios más hacía el este tienden a tener un índice más bajo (aunque en menos medida), influenciados por un alto % Población de 15 años o más analfabeta, un alto % Población  de 15 años o más sin educación básica, un alto % Viviendas particulares con hacinamiento, un alto % Población en localidades con menos de 5 000 habitantes o un alto % Población ocupada con ingresos menores a 2 salarios mínimos. Por tanto los datos hacía el Oeste tienden a tener un alto índice de marginación.

#### ISOMAP

In [255]:
iso = manifold.Isomap(n_neighbors=6, n_components=2)
proj = iso.fit_transform(data_norm)
porj_comp_plot_by_state(source_data, proj[:,0:2], title="ISOMAP por estados")

En este caso podemos ver una mejor calificación de los datos que en el PCA, donde están más separados por Estado, mostrando una correlación por este.

In [256]:
porj_comp_plot_by_imn(source_data, data_norm, proj[:,0:2], title="ISOMAP por índice de marginación")

También podemos ver que OVSDE, OVSEE, OVSAE siguen coincidiendo en la tendencia y relación que tienen estos como en el PCA, castigando entre más al norte te encuentres; así como ANALF y SBASC. Aún mas podemos ver como PL.5000 es como una unión de VHAC y PO2SM, estos junto con OVPT castigando entre más a al este te encuentres. Todo siguiendo las mismas "justificaciones" que en el PCA. También podemos notar una mayor correlación entre OVPT y VAHC.

Si nos fijamos en el IMN notaremos que este es casi inversamente proporcional a PO2SM, lo cuál tiene sentido puesto que este dictamina el salirio y economía de un municipio, el cuál es un factor muy importante en cuánto se pueda o no marginar una población.

Podemos ver que sigue una tendencia comoo el PCA donde Nuevo León tiene poco índice, Guanajuato medio índico y Chiapas un alto índice de marginación. Mientra más a la derecha esté un municipio este tendrá un menor índice de marginación.

#### T-SNE

In [257]:
tsne = TSNE(n_components=2, learning_rate='auto', early_exaggeration=12.0, perplexity=25, init='random')
proj = tsne.fit_transform(data_norm)
porj_comp_plot_by_state(source_data, proj[:,0:2], title="T-SNE por estados")

En el T-SNE pudimos encontrar una mejor agrupación de los municipios por estados, y reflejada con respecto al eje y con respecto a los dos anteriores algoritmos. Y conseguimos una dsitrbución un poco más espaciada de los datos para identificar mejor las zonas como lo es los estados.

También podemos observar que los datos parecieran están más asociados a las direciones noreste y suroeste.

In [258]:
porj_comp_plot_by_imn(source_data, data_norm, proj[:,0:2], title="T-SNE por índice de marginación")

En este caso podemos ver que PL.5000 es quien dictamina una mayor relación con IMN mientras más al este te encuentre, y es menos claro que este sea la unión de VHAC y PO2SM. Siguen coincidiendo OVSDE,OVSEE y OVSAE, y siguen teniendo una misma tendencia entre ellos. Caso similar con ANALF y SBASC siguen mostrando una misma dirección, aunque esta vez con pesos distintos.

Por lo visto anterior mente coincide con la gráfica reflejada con respecto al eje y con respecto a los dos anteriores algoritmo. Esto nos indica que un Municipio tiende a tener un mayor indice de marginación si este esta más al noroeste. También se cumple lo contrario.

### SOM

In [259]:
som = SOM(dim=9, m=3, n=3)
som.fit(data_norm.to_numpy())
label = som.predict(data_norm.to_numpy())

In [260]:
fig = make_subplots(rows=3, cols=3, subplot_titles=data_norm.columns)

for i,carac in enumerate(data_norm.columns):
    x=np.array([data_norm[carac].iloc[label==index].mean() for index in range(9)])
    fig.add_heatmap(
        row=int(i/3)+1,
        col=int(i%3)+1,
        z=((x-np.min(x))/(np.max(x)-np.min(x))).reshape((3,3)),
        coloraxis="coloraxis"
    )

fig.update_layout(
        template="simple_white",
        title="SOM",
        coloraxis=dict(colorscale='Viridis'),
        width=1000,
        height=1200
    )

fig.show()

En este caso volvieron a coincidir ANALF y SBASC, con muy parecidas coloraciones, relacionando mucho el analfabetismo con la carencia de educación basica. No es clara la relación en este caso con IVSDE, OVSEE, y OVSAE, pero en cambio si hay una mayor relación entre OVPT y VHAC. Por último la relación entre PL.500 y PO2SM se intensifico indicando una mayor relación entre la pobreza y el número de habitantes de su localidad.